In [620]:
# Lets us load the environment variables from the .env file
#!pip install python-dotenv

In [621]:
# Import libraries
import praw
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
from os import getenv
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [622]:
# Load .env file
load_dotenv('.env3')

True

In [623]:
# Load from the .env
load_dotenv()

# Get the environmental variables
APP_NAME = getenv('APP_NAME')
APP_ID = getenv("APP_ID")
APP_SECRET = getenv("APP_SECRET")
USERNAME = getenv('REDDIT_USERNAME')
PASSWORD = getenv('PASSWORD')

In [624]:
# Create a reddit instance using PRAW
import praw
reddit = praw.Reddit(client_id=APP_ID,
                     client_secret=APP_SECRET,
                     user_agent=APP_NAME,
                     username=USERNAME,
                     password=PASSWORD)


In [625]:
# Check that is loads correctly
USERNAME

'jaygilberts'

In [626]:
# Check that the variables all loaded
if APP_NAME and APP_ID and APP_SECRET and USERNAME and PASSWORD:
    print("Credentials loaded")
else:
    print("ERROR: Credentials not loaded!!")

Credentials loaded


In [627]:
# Library that let's us make http requests
import requests

# Sets up the authentication part of the request
auth = requests.auth.HTTPBasicAuth(f'{APP_ID}', f'{APP_SECRET}')

# Sets up the data we want to send: our login method (password), username, and password
data = {'grant_type': 'password',
        'username': f'{USERNAME}',
        'password': f'{PASSWORD}'}

# Sets up this requests header info, which gives reddit a brief description of our app
# This is the format requested by Reddit: os:app_name:version (by /u/username)
headers = {'User-Agent': f'windows:{APP_NAME}:v1.0 (by /u/{USERNAME})'}

# Makes the request to the access_token api endpoint, and saves the response in res
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# Converts response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# Adds authorisation to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# While the token is valid (~2 hours) we can just add headers=headers to our requests to prove authentication
# Making a test request. <Response [200]> means that it was a success!
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [628]:
# get 1000 hot posts from the Self Build subreddit
hot_posts = reddit.subreddit('selfbuild').hot(limit=1000)
for post in hot_posts:
    print(post.title)

How does claiming VAT on a self build work?
Hey Reddit Execs: stop being greedy assholes. This subreddit will go dark on Jun 12 permanently unless the 3rd party app fuckery is reversed
Come join our Facebook group!
Turn-key Timber house build onto unknown size plot
How to Mortgage a Self-Build House or Extension in the UK
How much did your whole project cost?
Buying a small house outright or building a nicer one with a plot of land ?
Advice for a newbie
Is this enough insulation
self build / mortgages in Ireland
Hi all, does anyone have a good excel budget template for self build? thanks!
2 Ways Permitted Development Could RUIN YOUR HOME
4 Things GUARANTEE your Planning Permission will be REFUSED
Plasterboard vs render
Barn conversion questions
What is the formula or online calculator for using wall depth and length and height to compute only the costs of the reinforced concrete for a house?
Windows & Doors - Triple vs Double Glazing
Can anyone help me design, or give any tips or thoug

In [629]:
# Define the keywords you want to look for
keywords = ["cost", "price", "budget", "", "estimate"]

In [630]:
# Create an empty dataframe
df = pd.DataFrame()

In [631]:
# Create an empty dataframe to store the results
df = pd.DataFrame(columns=["subreddit", "title", "url", "score", "num_comments", "keyword"])

In [632]:
# Loop through the subreddits
for sub in subreddits:
    # Get the top 100 posts from the subreddit
    posts = reddit.subreddit(sub).top(limit=1000)

In [633]:
 # Loop through each post
for post in posts:
        # Loop through each keyword
        for keyword in keywords:
            # Check if the keyword is in the post title
            if keyword.lower() in post.title.lower():
                # Append the post data to the dataframe
                df = df.append({"subreddit": sub,
                                "title": post.title,
                                "url": post.url,
                                "score": post.score,
                                "num_comments": post.num_comments,
                                "keyword": keyword}, ignore_index=True)

# Print the dataframe
print(df)

    subreddit                                              title  \
0       AskUK  Did anyone else dream of moving to America whe...   
1       AskUK  My girlfriend just ate a digestive with cheese...   
2       AskUK     Why is everyone so anti drug but pro alchohol?   
3       AskUK  Is anyone else feeling unsafe today because of...   
4       AskUK            What are peoples thoughts on the queue?   
..        ...                                                ...   
995     AskUK  Would you prefer permanent British Summer Time...   
996     AskUK  What's the worse country you've visited outsid...   
997     AskUK  Has anyone been able to ever befriend a Chines...   
998     AskUK  Why don’t we have deposits on drink cans in th...   
999     AskUK  What common English phrases might be lost in t...   

                                                   url  score num_comments  \
0    https://www.reddit.com/r/AskUK/comments/jnj3bq...  18200         2315   
1    https://www.reddit.com

In [634]:
# Loop through the subreddits
for sub in subreddits:
    # Get the top 100 posts from the subreddit
    posts = reddit.subreddit(sub).top(limit=1000)

In [635]:
import pandas as pd
posts = []
ml_subreddit = reddit.subreddit('selfbuild')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
print(posts)

                                                 title  score       id  \
0          How does claiming VAT on a self build work?      3  140iwyg   
1    Hey Reddit Execs: stop being greedy assholes. ...      1  140ov7q   
2                        Come join our Facebook group!      2  13v5vmh   
3    Turn-key Timber house build onto unknown size ...      3  13q874m   
4    How to Mortgage a Self-Build House or Extensio...      2  13psjll   
..                                                 ...    ...      ...   
116                                    BIFL self build      1   2euce6   
117                       Self build Q&A for beginners      3   2ea3zf   
118           Some good info for a self build beginner      3   2e9vly   
119                       Some nice looking eco builds      3   2e9v9y   
120                          Another FAQ for beginners      1   2ea5bm   

     subreddit                                                url  \
0    selfbuild  https://www.reddit.com/r/s

In [636]:
 # Loop through the posts
for post in posts:
        # Check if any of the keywords are in the title
        if any(word in post.title.lower() for word in keywords):
            # Calculate the sentiment score using a simple formula
            sentiment = post.score * (post.upvote_ratio - 0.5)

AttributeError: 'builtin_function_or_method' object has no attribute 'lower'

In [ ]:
   # Append the post data to the dataframe
df = df.append({'subreddit': sub,
                            'title': post.title,
                            'score': post.score,
                            'sentiment': sentiment}, ignore_index=True)

# Print the dataframe
print(df)


In [ ]:
# Define a function to get the cost of self build from Reddit posts
def get_cost(post):
    # Extract the text from the post
    text = post.selftext
    # Use a regular expression to find a number followed by a currency symbol
    pattern = r'\d+(?:\.\d+)?\s*[£€$]'
    # Search for the pattern in the text
    match = re.search(pattern, text)
    # If a match is found, return the cost as a float
    if match:
        cost = match.group()
        cost = cost.replace('£', '')
        cost = cost.replace('€', '')
        cost = cost.replace('$', '')
        cost = float(cost)
        return cost
    # Otherwise, return None
    else:
        return None

In [ ]:
# Create a list to store the costs of self build
costs = []

In [ ]:
# Append the post data to the dataframe
df = df.append({'subreddit': sub,
                            'title': post.title,
                            'score': post.score,
                            'sentiment': sentiment}, ignore_index=True)



In [637]:
# Print the dataframe
print(df)

    subreddit                                              title  \
0       AskUK  Did anyone else dream of moving to America whe...   
1       AskUK  My girlfriend just ate a digestive with cheese...   
2       AskUK     Why is everyone so anti drug but pro alchohol?   
3       AskUK  Is anyone else feeling unsafe today because of...   
4       AskUK            What are peoples thoughts on the queue?   
..        ...                                                ...   
995     AskUK  Would you prefer permanent British Summer Time...   
996     AskUK  What's the worse country you've visited outsid...   
997     AskUK  Has anyone been able to ever befriend a Chines...   
998     AskUK  Why don’t we have deposits on drink cans in th...   
999     AskUK  What common English phrases might be lost in t...   

                                                   url  score num_comments  \
0    https://www.reddit.com/r/AskUK/comments/jnj3bq...  18200         2315   
1    https://www.reddit.com

In [638]:
# Write more code to gather information about costs of self build for uk and ireland using reddit api and python

# Define the base url for reddit api
base_url = "https://www.reddit.com/"


In [639]:
# Define the headers for reddit api
headers = {"User-agent": "python:costs_of_self_build:v1.0"}

In [640]:
# Define the subreddits to search
subreddits = ["northernireland", "ireland", "AskUK"]

In [641]:
# Define the keywords to search
keywords = ["self build", "build a house", "building costs"]

In [642]:
# Create an empty list to store the results
results = []

In [643]:
# Loop through each subreddit
for sub in subreddits:

    # Loop through each keyword
    for keyword in keywords:

        # Define the search url with parameters
        search_url = base_url + "r/" + sub + "/search.json?q=" + keyword + "&restrict_sr=1"

        # Make a get request to the search url
        response = requests.get(search_url, headers=headers)

        # Check if the response is successful
        if response.status_code == 200:

            # Parse the json data from the response
            data = response.json()

            # Get the list of posts from the data
            posts = data["data"]["children"]

            # Loop through each post
            for post in posts:

                # Get the post details
                title = post["data"]["title"]
                score = post["data"]["score"]
                url = post["data"]["url"]

                # Append the post details to the results list
                results.append({"subreddit": sub,
                                "keyword": keyword,
                                "title": title,
                                "score": score,
                                "url": url})

# Create a pandas dataframe from the results list
import pandas as pd

df = pd.DataFrame(results)

# Print the dataframe
print(df)


           subreddit         keyword  \
0    northernireland      self build   
1    northernireland      self build   
2    northernireland      self build   
3    northernireland      self build   
4    northernireland      self build   
..               ...             ...   
214            AskUK  building costs   
215            AskUK  building costs   
216            AskUK  building costs   
217            AskUK  building costs   
218            AskUK  building costs   

                                                 title  score  \
0                                    Self build advice      5   
1                              Self Build Co-Operative      2   
2                     Self build mortgage, which bank?      5   
3                           Self build N. I LAND SITES      0   
4                    Should I hold off buying a house?     14   
..                                                 ...    ...   
214  Can someone translate this solicitor mumbo jum...      2   

In [644]:
# Import NLTK and VADER sentiment analyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [645]:
# Download NLTK data if not already installed
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jaygi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [646]:
# Initialize sentiment analyzer
sid = SentimentIntensityAnalyzer()


In [647]:
# Define a function to get sentiment scores
def get_sentiment_scores(text):
    # Get sentiment scores for each sentence in the text
    scores = sid.polarity_scores(text)
    # Return a dictionary with the average scores for each sentiment
    return {'neg': scores['neg'],
            'neu': scores['neu'],
            'pos': scores['pos'],
            'compound': scores['compound']}

In [648]:
# Apply the function to the title column of the dataframe
df['sentiment'] = df['title'].apply(get_sentiment_scores)



In [649]:
# Print the dataframe with sentiment scores
print(df)

           subreddit         keyword  \
0    northernireland      self build   
1    northernireland      self build   
2    northernireland      self build   
3    northernireland      self build   
4    northernireland      self build   
..               ...             ...   
214            AskUK  building costs   
215            AskUK  building costs   
216            AskUK  building costs   
217            AskUK  building costs   
218            AskUK  building costs   

                                                 title  score  \
0                                    Self build advice      5   
1                              Self Build Co-Operative      2   
2                     Self build mortgage, which bank?      5   
3                           Self build N. I LAND SITES      0   
4                    Should I hold off buying a house?     14   
..                                                 ...    ...   
214  Can someone translate this solicitor mumbo jum...      2   